# Worflow to train the ML model - ATI interview

This workflow performs the following tasks:
- download the substrate structures from materials project
- generate the adsorbate structures (pymatgen)
- generate the adsorption structures
- write the CRYSTAL inputs
- run the adsorbate+substrate CRYSTAL calculations
- run the adsorbate and substrate CRYSTAL calculations (GHOST atoms)
- calculate the E_ads (BSSE corrected)

In [16]:
# crystal_functions imports
from file_readwrite import write_cry_input, write_cry_gui
from file_readwrite import Crystal_input, Crystal_output
from adsorb import sub_ads_indices
from calculate import cry_ads_energy
from execute import runcry

# pymatgen imports
from pymatgen.core.structure import Molecule, Structure, Lattice
from pymatgen.core.surface import SlabGenerator, generate_all_slabs
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
from pymatgen.ext.matproj import MPRester

# other imports
import shutil
import pandas as pd
import numpy as np

### Make adsorbate - make toppings
Make the adsorbate structures using the Molecule class in pymatgen

In [17]:
mushroom = Molecule('O',[[0.0, 0.0, 0.0]])
ham = Molecule('CO',[[0.0, 0.0, 0.0],[ 0.0, 0.0, 1.128]])
olive = Molecule('HHO',[[0.76,0.00,0.50],[0.76, 0.00,-0.50],[0.0, 0.0, 0.0]])
adsorbates = [mushroom,ham,olive]

### Make substrates - make pizza base

#### Bulk
Download the bulk structures from the Materials Project

In [18]:
with MPRester("p5vAQV3F1QuxFcxVT") as m:    
    normal = m.get_structure_by_material_id("mp-30")
    whole_wheat = m.get_structure_by_material_id("mp-1265")
    gluten_free = m.get_structure_by_material_id("mp-2657")
    ny_style = m.get_structure_by_material_id("mp-22862")
materials = [normal,whole_wheat,gluten_free,ny_style]

Ensure the conventional cell is used

In [19]:
bulks = []
substrates = []
for material in materials:
    bulks.append(SpacegroupAnalyzer(material).get_conventional_standard_structure())

#### Slab
Generate the slabs using the pymatgen SlabGenerator function.

In [20]:
substrates =[]
for bulk in bulks:
    slabs = generate_all_slabs(bulk, max_index=1, min_slab_size=2., min_vacuum_size=10.0, 
                                   center_slab=False, symmetrize=True, in_unit_planes=False, max_normal_search=5) 
    substrates.extend(slabs)

### Adsorb - place toppings on pizza
Place the adsorbate on the top surfaces of the slab.

In [21]:
sub_composition = []
ads_composition = []
miller_indices = []    
n_layers = []
sub_ads_structures = []
for substrate in substrates:
    for adsorbate in adsorbates:
        adsorption_structures = AdsorbateSiteFinder(substrate).generate_adsorption_structures(adsorbate,repeat=[1,1,1])
        sites = AdsorbateSiteFinder(substrate).find_adsorption_sites()    
        for i,adsorption_structure in enumerate(adsorption_structures):
            miller_indices.append(adsorption_structure.miller_index)
            n_layers.append(len(np.unique(substrate.cart_coords[:,2])))  
            sub_composition.append(substrate.composition.reduced_formula)
            ads_composition.append(str(adsorbate.composition.hill_formula).replace(" ", ""))
            sub_ads_structures.append(adsorption_structure)

### Prepare inputs (geometry optimisation)
Define CRYSTAL input parameters.

In [22]:
geom_block = ['Adsorption tests\n','EXTERNAL\n','EXTPRT\n','OPTGEOM\n','END\n']
bs_block = ['BASISSET\n', 'STO-3G\n']
func_block = ['DFT\n', 'B3LYP\n', 'XXLGRID\n', 'ENDDFT\n']
scf_block = [['TOLINTEG\n', '5 5 5 5 10\n'],
             ['SHRINK\n', '6 12\n'],
             ['MAXCYCLE\n', '20\n'],
             ['FMIXING\n', '70\n'],
             'DIIS\n',
             'ENDSCF\n']

#### Write inputs
Write the inputs to file.

In [23]:
file_names = []
for i,structure in enumerate(sub_ads_structures):
    input_name = 'data/'+str(sub_composition[i]).replace(" ", "")+'_'+str(ads_composition[i]).replace(" ", "")+'_' \
               +''.join(str(x) for x in structure.miller_index)+'_'+str(i+1)+'.d12'
    file_names.append(input_name[:-4])
    write_cry_input(input_name,crystal_blocks=[geom_block,bs_block,func_block,scf_block],external_obj=structure)

### Run the calculations
Use the crystal_functions runcry function to execute CRYSTAL (please ensure the path to your runcry17 is defined in execute.runcry()

In [26]:
for cry_input in file_names:
    runcry(cry_input)
    pass

### Read the final energy and band gap
Use the crystal_functions Crystal_output class to extract the final energy and band gap.

In [95]:
E_full_system = []
fermi_energy = []
for cry_input in file_names:
    cry_output = Crystal_output(cry_input+'.out')
    if cry_output.converged == True:
        E_full_system.append(cry_output.final_energy()) 
        fermi_energy.append(cry_output.fermi_energy())

### Prepare inputs for the BSSE calculations
Generate the inputs for the BSSE calculation. The indices of the GHOST atoms are obtained from the pymatgen.core.structure.Slab object ('adsorbate' and 'substrate').

In [70]:
for i,cry_input in enumerate(file_names):
    
    #Substrate        
    bsse_sub_inp = Crystal_input(cry_input+'.d12')
    bsse_sub_inp.add_ghost(indices['adsorbate'])
    bsse_sub_inp.opt_to_sp()
    bsse_sub_inp_name = cry_input+'_BSSE_sub.d12'
    write_cry_input(bsse_sub_inp_name,bsse_sub_inp)

    shutil.copy(cry_input+'.gui',cry_input+'_BSSE_sub.gui')

    #Adsorbate
    bsse_ads_inp = Crystal_input(cry_input+'.d12')
    bsse_ads_inp.add_ghost(indices['substrate'])
    bsse_sub_inp.opt_to_sp()
    bsse_ads_inp_name = cry_input+'_BSSE_ads.d12'
    write_cry_input(bsse_ads_inp_name,bsse_ads_inp)

    shutil.copy(cry_input+'.gui',cry_input+'_BSSE_ads.gui')

### Run the BSSE calculations
Use the crystal_functions runcry function to execute CRYSTAL (please ensure the path to your runcry17 is defined in execute.runcry()

In [98]:
E_sub_BSSE = []
E_ads_BSSE = []
for i,cry_input in enumerate(file_names):
    runcry(cry_input+'_BSSE_sub')
    cry_BSSE_sub_output = Crystal_output(cry_input+'_BSSE_sub.out')
    if cry_BSSE_sub_output.converged == True:
        E_sub_BSSE.append(cry_BSSE_sub_output.final_energy())
    else:
        E_sub_BSSE.append(None)
        
    runcry(cry_input+'_BSSE_ads')
    cry_BSSE_ads_output = Crystal_output(cry_input+'_BSSE_ads.out')
    if cry_BSSE_ads_output.converged == True:
        E_ads_BSSE.append(cry_BSSE_ads_output.final_energy())
    else:
        E_ads_BSSE.append(None)

### Calculate the adsorption energy
Use the crystal_functions cry_ads_energy function to get the adsorption energy (BSSE corrected).

In [100]:
E_adsorption = []
for i in range(len(file_names)):
    E_adsorption.append(cry_ads_energy(E_full_system[i],E_sub_BSSE[i],E_ads_BSSE[i]))

### Create Dataframe

In [112]:
df = pd.DataFrame(list(zip(file_names, sub_composition, n_layers, ads_composition,  miller_indices, E_adsorption,
                           fermi_energy,E_sub_BSSE,E_ads_BSSE,E_full_system)),
               columns =['File name', 'Substrate','N layers','Adsorbate','Miller Indices','E adsorption (BSSE)',
                         'Fermi energy','E sub (ads ghost)','E ads (ads ghost)','E full system'])
df

,File name,Substrate,N layers,Adsorbate,Miller Indices,E adsorption (BSSE),Fermi energy,E sub (ads ghost),E ads (ads ghost),E full system
0,data/Cu_O_111_1,Cu,3,O,"(1, 1, 1)",4420.730098,-21.413853,-131572.411437,-3039.437924,-130191.119262
1,data/Cu_O_111_2,Cu,3,O,"(1, 1, 1)",5442.876448,-26.251543,-130710.870443,-2047.155883,-127315.149878
2,data/Cu_O_111_3,Cu,3,O,"(1, 1, 1)",4420.730098,-21.413853,-131572.411437,-3039.437924,-130191.119262
3,data/Cu_O_111_4,Cu,3,O,"(1, 1, 1)",4420.730098,-21.413853,-131572.411437,-3039.437924,-130191.119262
4,data/Cu_CO_111_5,Cu,3,CO,"(1, 1, 1)",5442.876448,-26.251543,-130710.870443,-2047.155883,-127315.149878
...,...,...,...,...,...,...,...,...,...,...
406,data/NaCl_CO_100_407,NaCl,2,CO,"(1, 0, 0)",4420.730098,-21.413853,-131572.411437,-3039.437924,-130191.119262
407,data/NaCl_H2O_100_408,NaCl,2,H2O,"(1, 0, 0)",4420.730098,-21.413853,-131572.411437,-3039.437924,-130191.119262
408,data/NaCl_H2O_100_409,NaCl,2,H2O,"(1, 0, 0)",4420.730098,-21.413853,-131572.411437,-3039.437924,-130191.119262
409,data/NaCl_H2O_100_410,NaCl,2,H2O,"(1, 0, 0)",4420.730098,-21.413853,-131572.411437,-3039.437924,-130191.119262


### Display lower Fermi Energy

In [113]:
df[df['Fermi energy'] < -26.0]

,File name,Substrate,N layers,Adsorbate,Miller Indices,E adsorption (BSSE),Fermi energy,E sub (ads ghost),E ads (ads ghost),E full system
1,data/Cu_O_111_2,Cu,3,O,"(1, 1, 1)",5442.876448,-26.251543,-130710.870443,-2047.155883,-127315.149878
4,data/Cu_CO_111_5,Cu,3,CO,"(1, 1, 1)",5442.876448,-26.251543,-130710.870443,-2047.155883,-127315.149878
11,data/Cu_H2O_111_12,Cu,3,H2O,"(1, 1, 1)",5442.876448,-26.251543,-130710.870443,-2047.155883,-127315.149878
22,data/Cu_H2O_110_23,Cu,4,H2O,"(1, 1, 0)",5442.876448,-26.251543,-130710.870443,-2047.155883,-127315.149878
23,data/Cu_H2O_110_24,Cu,4,H2O,"(1, 1, 0)",5442.876448,-26.251543,-130710.870443,-2047.155883,-127315.149878
...,...,...,...,...,...,...,...,...,...,...
389,data/NaCl_CO_110_390,NaCl,5,CO,"(1, 1, 0)",5442.876448,-26.251543,-130710.870443,-2047.155883,-127315.149878
390,data/NaCl_CO_110_391,NaCl,5,CO,"(1, 1, 0)",5442.876448,-26.251543,-130710.870443,-2047.155883,-127315.149878
391,data/NaCl_CO_110_392,NaCl,5,CO,"(1, 1, 0)",5442.876448,-26.251543,-130710.870443,-2047.155883,-127315.149878
393,data/NaCl_H2O_110_394,NaCl,5,H2O,"(1, 1, 0)",5442.876448,-26.251543,-130710.870443,-2047.155883,-127315.149878


### Display NaCl

In [114]:
df[df['Substrate'] == 'NaCl']

,File name,Substrate,N layers,Adsorbate,Miller Indices,E adsorption (BSSE),Fermi energy,E sub (ads ghost),E ads (ads ghost),E full system
381,data/NaCl_O_110_382,NaCl,5,O,"(1, 1, 0)",4420.730098,-21.413853,-131572.411437,-3039.437924,-130191.119262
382,data/NaCl_O_110_383,NaCl,5,O,"(1, 1, 0)",4420.730098,-21.413853,-131572.411437,-3039.437924,-130191.119262
383,data/NaCl_O_110_384,NaCl,5,O,"(1, 1, 0)",4420.730098,-21.413853,-131572.411437,-3039.437924,-130191.119262
384,data/NaCl_O_110_385,NaCl,5,O,"(1, 1, 0)",4420.730098,-21.413853,-131572.411437,-3039.437924,-130191.119262
385,data/NaCl_O_110_386,NaCl,5,O,"(1, 1, 0)",4420.730098,-21.413853,-131572.411437,-3039.437924,-130191.119262
386,data/NaCl_O_110_387,NaCl,5,O,"(1, 1, 0)",4420.730098,-21.413853,-131572.411437,-3039.437924,-130191.119262
387,data/NaCl_CO_110_388,NaCl,5,CO,"(1, 1, 0)",4420.730098,-21.413853,-131572.411437,-3039.437924,-130191.119262
388,data/NaCl_CO_110_389,NaCl,5,CO,"(1, 1, 0)",4420.730098,-21.413853,-131572.411437,-3039.437924,-130191.119262
389,data/NaCl_CO_110_390,NaCl,5,CO,"(1, 1, 0)",5442.876448,-26.251543,-130710.870443,-2047.155883,-127315.149878
390,data/NaCl_CO_110_391,NaCl,5,CO,"(1, 1, 0)",5442.876448,-26.251543,-130710.870443,-2047.155883,-127315.149878
